![alt text](https://www.ctdb.hcmus.edu.vn/wp-content/uploads/2017/04/17880008_1006985519432248_7695519039665360507_o-1.jpg)

Chắc hẳn ai trong chúng ta cũng từng gặp một tấm poster như thế này trong đời. Ít nhất thì đây là lần đầu tiên nếu trước đó bạn chưa thấy.

Có thể vì tính thẩm mỹ, tiết kiệm không gian hoặc vì một lý do nào đó không biết được, chúng ta chỉ thấy được các logo của công ty trên tấm poster này.

Vậy, bạn có thể kể hết tên các công ty có mặt trên poster này không. Tôi đoán đa phần là không, ít nhất thì tôi không biết.

Xuất phát từ nhu cầu muốn biết công ty nào được in trên các poster của chính chúng tôi và giúp nhưng người có nhu cầu giống như thế, chúng tôi muốn phát triển một hệ thống có thể nhận biết được công ty thông qua logo của họ.

Do thời gian có hạn và tài nguyên không đủ, chúng tôi chỉ làm một ví dụ với 5 công ty là vng, viettel, fpt, kms, fujinet. Chúng tôi sẽ phát triển thêm trong tương lại khi gọi vốn thành công từ các sharks :)). Dữ liệu của chúng tôi được thu thập từ internet. Cụ thể là từ google images. Chúng tôi chỉ lấy những tấm ảnh được gán nhán là được phép sử dụng (free).

Cuối cùng, nếu bạn thích ý tưỡng này, hãy đầu tư cho chúng tôi. Xin cảm ơn.

# Import library

In [0]:
from requests_html import HTMLSession
import requests
import os
import argparse
import sys
import json
import time

In [0]:
keywords = {
    'viettel': ['cua hang viettel', 'viettel store'],
    'fpt': ['fpt software', 'fpt'],
    'fujinet': ['fujinet', 'fujinet logo'],
    'vng': ['vng', 'vng logo'],
    'kms': ['kms', 'kms logo'],
}

In [0]:
max_image_per_query = 100
image_dir_root = './download'
if not os.path.exists(image_dir_root):
    os.mkdir(image_dir_root)

In [0]:
session = HTMLSession()

In [0]:
def fetch_image_to_dir(query, output_dir, start_index=0, verbose=False):
    query= query.split()
    query='+'.join(query)

    url="https://www.google.co.in/search?q="+query+"&source=lnms&tbm=isch&tbs=sur:f"
    res = session.get(url)

    while not(res.ok):
        time.sleep(1)
        res = session.get(url)

    tmp = res.html.find('.rg_meta')
    print(f'len {len(tmp)} res.ok {res.ok} url {url}')

    i = start_index
    for value in res.html.find('.rg_meta'):
        json_data = json.loads(value.text)
        img = json_data['ou']
        ext = json_data['ity']

        if ext == '':
            continue
    
        if verbose:
            print(f'Downloading i={i}: {img}')

        try:
            raw_img = requests.get(img, timeout=10).content
            f = open(os.path.join(output_dir, f'img_{i}.{ext}'), 'wb')
            f.write(raw_img)
            f.close()
        except Exception as e:
            print(e)

        i += 1
        if (i - start_index) == max_image_per_query:
            break
    return i

In [0]:
for company, queries in keywords.items():
    img_dir = os.path.join(image_dir_root, company)
    if not os.path.exists(img_dir):
        os.mkdir(img_dir)
        
    index = 0
    for query in queries:
        index = fetch_image_to_dir(query, img_dir, start_index=index, verbose=True)

# View some results